In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from nba_api.stats.endpoints import commonallplayers, playerindex

player_dict = get_players_pbp()
teams_dict, teams_list = get_teams()

data_DIR = "../data/all_in_one_metrics/"
data_DIR1 = "../data/injuries/"
box_DIR = "../data/box/"
fig_DIR = "../figs/team_leaders/"
shiny_DIR = "../data/shiny/"

In [ ]:
def is_injured(dfinj, pId_missed, game_date):
    missed_games = np.array([False] * len(pId_missed))
    for i,pId in enumerate(pId_missed):
        df_p = dfinj.query(f'playerID == {pId}').reset_index(drop=True)
        if len(df_p) > 0:
            df_p["Comp"] = df_p["Date"] <= game_date
            idxi = df_p[df_p["Comp"]].index
            if len(idxi) > 0:
                idx = idxi[-1]
                missed_game = df_p["Out"].loc[idx]
                missed_games[i] = missed_game
    gp = missed_games*pId_missed
    pId_m = gp[gp !=0 ]
    # pId_m = list(pId_m)
    return pId_m

In [ ]:
year = 2023

In [ ]:
season_str = str(year) + "-" + str(year+1)[-2:]
df0 = pd.read_parquet(box_DIR + f"NBA_BOX_T_Base_{year}.parquet")
df0= df0[["GAME_ID","TEAM_ID","GAME_DATE","MATCHUP","WL","PLUS_MINUS"]]
df0["GAME_ID"] = df0["GAME_ID"].astype(int)
# load player indvidual game boxscores
df1 = pd.read_parquet(box_DIR + "NBA_BOX_P_" + "Base" + "_" + str(year) + ".parquet")
df1["GAME_DATE"] = pd.to_datetime(df1["GAME_DATE"], format="%Y-%m-%d")
dfinj = pd.read_parquet(data_DIR1 + f'NBA_prosptran_injuries_{year}.parquet')
player_list = df1["PLAYER_ID"].unique()
dfr = df1[["TEAM_ID","PLAYER_ID"]]
dfrt = dfr.groupby("TEAM_ID")
df2 = df1.groupby(["GAME_ID","TEAM_ID"])
# loop through all groups and get injured players 
keys = list(df2.groups)
dfma = []
# i=0
for key in tqdm(keys):
    p_played =  df2["PLAYER_ID"].get_group(key).to_numpy()
    p_roster = dfrt["PLAYER_ID"].get_group(key[1]).to_numpy()
    # pId_missed = np.array(list(set(p_roster) - set(p_played)))#.astype(int)
    pId_missed = np.setdiff1d(p_roster,p_played)
    game_date = df2["GAME_DATE"].get_group(key).iloc[0]
    players_missed = is_injured(dfinj,pId_missed,game_date)
    dfm1 = pd.DataFrame({"PLAYER_ID":players_missed})
    dfm1["TEAM_ID"] = key[1]
    dfm1["GAME_ID"] = key[0]
    dfm1["GAME_DATE"] = game_date
    dfma.append(dfm1)
    # i+=1
    # if i > 50:
    #     break
dfm1 = pd.concat(dfma)
dfm1["PLAYER_ID"] = dfm1["PLAYER_ID"].astype(int)
# map player and team names
dfm1["PLAYER_NAME"] = dfm1["PLAYER_ID"].map(player_dict)
dfm1["TEAM_NAME"] = dfm1["TEAM_ID"].map(teams_dict)


In [ ]:
dflb = pd.read_csv(data_DIR + f"NBA_LEBRON_{year}.csv")
dflb = dflb.rename(columns={"LEBRON_WinsAdded":"LEBRON_WAR"})
dflb = dflb[["PLAYER_ID","LEBRON_WAR"]]
dfgp = pd.read_parquet(box_DIR + f"NBA_BOX_P_Cum_Base_{year}.parquet")
dfgp = dfgp[["PLAYER_ID","GP","MIN"]]
dflm = pd.merge(dflb,dfgp,on="PLAYER_ID")
dflm["LBWAR_PG"] = round(dflm["LEBRON_WAR"]/dflm["GP"],4)
dflm = dflm[["PLAYER_ID","LBWAR_PG","MIN"]]

In [ ]:
dfm = pd.merge(dfm1,dflm,on="PLAYER_ID")

In [ ]:
dfmg =  (
    dfm
    .groupby(["GAME_ID","TEAM_ID"])[["PLAYER_ID","LBWAR_PG","MIN"]]
    .agg({"PLAYER_ID":["count"],"MIN":["sum"],"LBWAR_PG":["sum"]})
)
dfmg.columns = ["Games_Missed","Minutes_Missed","LEBRON_WAR_Missed"]
dfmg = dfmg.reset_index()
dfmg["GAME_ID"] = dfmg["GAME_ID"].astype(int)
# merge team boxscores to get game details like date and matchup
dfmf = pd.merge(df0,dfmg,on=["GAME_ID","TEAM_ID"])
dfmf["Team"] = dfmf["TEAM_ID"].map(teams_dict)
dfmf = dfmf.drop(columns=["GAME_ID","TEAM_ID"])
dfmf.insert(1,"Team",dfmf.pop("Team"))
df_x = dfmf.groupby("Team").agg({"Games_Missed":["sum"],"Minutes_Missed":["sum"],"LEBRON_WAR_Missed":["sum"]})
df_x.columns = ["Games_Missed","Minutes_Missed","LEBRON_WAR_Missed"]
df_x = (df_x
        .reset_index()
        .sort_values("LEBRON_WAR_Missed",ascending=False)
        .reset_index(drop=True)
    )
df_x["LEBRON_WAR_Missed"] = df_x["LEBRON_WAR_Missed"].round(2)

In [ ]:
df_teams = pd.read_csv("../data/" + "NBA_teams_colors_logos.csv")
df_teams["Team"] = df_teams["nameTeam"]
df_teams = df_teams[["Team","colorsTeam"]]
df_y = pd.merge(df_x, df_teams, on="Team")

In [ ]:
df_avg = df_y.iloc[:,1:-1].mean()
df_avg = pd.DataFrame(df_avg).T
df_avg["Games_Missed"] = df_avg["Games_Missed"].round(0)
df_avg["Minutes_Missed"] = df_avg["Minutes_Missed"].round(1)
df_avg["LEBRON_WAR_Missed"] = df_avg["LEBRON_WAR_Missed"].round(2)
df_avg["Team"] = "League Average"
df_avg["colorsTeam"]  = "#000000"

In [ ]:
df_z = pd.concat([df_y,df_avg]).sort_values("LEBRON_WAR_Missed",ascending=False).reset_index(drop=True)

In [ ]:
teams = df_z["Team"].to_list()
teams.reverse()
df_z["Team"] = pd.Categorical(df_z['Team'], categories=teams)
# df_y.to_parquet(shiny_DIR + "NBA_games_minutes_war_missed.parquet")

In [ ]:
%reload_ext rpy2.ipython

In [ ]:
%%R -i df_x
df <- df_x
library(tidyverse)
library(gt)
library(gtExtras)
df %>% 
  gt()%>%
  tab_header(
    title = md("**LEBRON WAR Missed due to Injury/Suspensions etc. 2023-24**"),
    subtitle = "Only Players who have played this season are considered"  
    ) %>%
    cols_label( index="#", LEBRON_WAR_OUT="LEBRON WAR"
    ) %>%
    gt_plt_bar(column = LEBRON_WAR_OUT, keep_column = TRUE, color = "red") %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 12,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 10,
        heading.title.font.size = 14,
        heading.subtitle.font.size = 12,
        table.font.names = "Consolas", 
        data_row.padding = px(0.5),
        table.margin.left = 1,
        table.margin.right = 1,

    ) %>% 
    tab_source_note(
    source_note = "@SravanNBA | Source: @knarsu3, bball-index, nba" ) %>% gtsave("../figs/team_leaders/lebron_war_missed.png",zoom=5) 